In [295]:
words = open('names.txt', 'r').read().splitlines()

In [296]:

words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [297]:

len(words)

32033

In [298]:

min(len(w) for w in words)

In [299]:

max(len(w) for w in words)

In [300]:
# Assuming `words` is your list of names
long_names = [name for name in words if len(name) == 15]


In [301]:
for name in long_names:
    print(name)


In [302]:
for w in words[:1]:
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1,ch2 in zip(w, w[1:]):
        print(ch1,ch2)

In [303]:
my_string = "hello"
my_list_of_chars = list(my_string)
print(my_list_of_chars+ list("this is awesome"))


In [304]:

b= {}

for w in words:
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1,ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)   
        b[bigram] = b.get(bigram, 0) + 1
        #print(ch1,ch2)

In [305]:
sorted(b.items(), key = lambda kv: kv[1])

In [306]:
def get_count(kv):
    return -kv[1]

sorted_b = sorted(b.items(), key=get_count)
sorted_b


In [307]:
#store this info in 2d array
import torch



In [308]:
a= torch.zeros((3,5), dtype=torch.int32)
a

In [309]:
a.dtype

In [310]:
a[1,3] =3

In [311]:
a

In [312]:
a[0,0] = 5

In [313]:
N = torch.zeros((27,27), dtype=torch.int32)


In [314]:
chars = sorted(list(set(''.join(words))))
stoi ={s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}; 
stoi

In [315]:

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1
        #print(ch1,ch2)

In [316]:
import matplotlib.pyplot as plt
%matplotlib inline

In [317]:
plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off');

In [318]:
p = N[0].float(); p

In [319]:
P = N.float()
P.sum(1, keepdim =True).shape

P = N.float()
P /= P.sum(1, keepdim = True)

In [320]:
P[0].sum()

In [321]:

g = torch.Generator().manual_seed(2147483647)

for i in range(50):
    out = []
    ix = 0
    while True:
        p = P[ix]
#         p = torch.ones(27)/27.0
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break

    print("".join(out))

In [322]:
# create the training set of all the bigrams
xs, ys = [],[]
for w in words[:1]:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)



In [323]:
xs




In [324]:
ys

In [325]:
import torch.nn.functional as F

In [326]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=27).float()
xenc

In [327]:
plt.imshow(xenc)
xenc.shape

In [328]:
W = torch.randn((27, 27), requires_grad = True)

In [329]:
xenc @ W

In [330]:
counts = (xenc @ W).exp()
probs = counts / counts.sum(1, keepdims=True)
probs
loss = -probs[torch.arange(5) ,ys].log().mean()

In [331]:
probs[0].shape

In [332]:
probs[0]

In [333]:
loss = -probs[torch.arange(5) ,ys].log().mean()

In [334]:
loss.item()

In [335]:
W.grad = None # set to zero
loss.backward()


In [336]:
W.grad.shape

In [337]:
W.data += -0.1*W.grad

In [338]:
import torch



# Prepare data
xs, ys = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

# Initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)


In [339]:
(W**2).sum()

In [340]:
# Gradient descent
for k in range(100):
    
    # Forward pass
    xenc = F.one_hot(xs, num_classes=27).float()  # Input to the network: one-hot encoding
    logits = xenc @ W  # Predict log-counts
    counts = logits.exp()  # Counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True)  # Probabilities for next character
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean() + 0.01*(W**2).mean()
    print(loss.item())
    
    # Backward pass
    W.grad = None  # Set to zero the gradient
    loss.backward()
    
    # Update
    W.data += -50 * W.grad  # Update weights with learning rate of 50


In [341]:
for i in range(5):
    out = []
    ix = 0
    while True:
        
        # Predict next character probabilities
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = xenc @ W  # Predict log-counts
        counts = logits.exp()  # Counts, equivalent to N
        p = counts / counts.sum(1, keepdims=True)  # Probabilities for next character
        
        # Sample from the distribution
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    
    print(''.join(out))